This notebook is used for neatly structuring all results and saving them to excel files, which can later be used in the report. The report is somewhat structured based on the sections for which results are generated. However, it probably is not possible to use as a standalone file due to specific ways in which data has to be structu

In [1]:
import pandas as pd
import json
import os
import pickle
from sklearn.metrics import confusion_matrix

In [2]:
path_res_rl = r"C:\Users\s159655\Documents\JADS\Thesis\RESULTS\Results_Paper\rl_results\all_results"
path_conf_rl = r"C:\Users\s159655\Documents\JADS\Thesis\Code\Train_Test\Configurations\RL_configs"

path_res_ae = r"C:\Users\s159655\Documents\JADS\Thesis\RESULTS\Results_Paper\ae_results\all_results"
path_conf_ae = r"C:\Users\s159655\Documents\JADS\Thesis\Code\Train_Test\Configurations\AE_configs"

path_res_ma = r"C:\Users\s159655\Documents\JADS\Thesis\RESULTS\Results_Paper\ma_results\all_results"
path_conf_ma = r"C:\Users\s159655\Documents\JADS\Thesis\Code\Train_Test\Configurations\MA_configs"

path_res_uo = r"C:\Users\s159655\Documents\JADS\Thesis\RESULTS\Results_Paper\uo_results\all_results"
path_conf_uo = r"C:\Users\s159655\Documents\JADS\Thesis\Code\Train_Test\Configurations\UO_configs"

In [3]:
def get_result_config(path_res, path_conf, configuration_info=True, return_df=True):
    with open(path_res, 'r') as f:
        results = json.load(f)
    with open(path_conf, 'r') as f2:
        conf = json.load(f2)
    confname = os.path.basename(path_conf)
    bleu = results['BLEU_SCORES']
    bleu_1 = bleu['BLEU_1']
    bleu_2 = bleu['BLEU_2']
    bleu_3 = bleu['BLEU_3']
    bleu_4 = bleu['BLEU_4']

    # Main novelty measures
    novelty_percentage = results["novelty_score"] 
    fraction_distinct = results["fraction_Distinct"]
    within_item_duplicates = results["within_item_duplicates"]
    fraction_within_item_duplicates = results["fraction_within_duplicates"]
    vocab_percentage_from_train = results["vocab_perc_intrain"]

    # Secondary novelty measures
    nr_duplicates = results["nr_duplicate_total"]
    nr_distinct = results["nr_distinct"]
    vocab_size_predictions = results["vocab_size_predictions"]
    vocab_size_references = results["vocab_size_ref"]
    vocab_percentage_inref = results["vocab_perc_inref"]
    vocab_percentage_ref_intrain = results["vocab_per_test_train"]
    rouge_l = results["ROUGE_SCORES"]["Avg"]["rouge-l"]["f"]
    
    if configuration_info:
        emb = conf['embedding_name']
        feat_ex = conf['img_folder'].split("_")
        feat_ex = "_".join(feat_ex[:2])
        dropout = conf['dropout']
        opt = conf['optimizer']
        arc = conf['architecture_type']
        nodes = conf['nr_nodes']
        df = [confname[:-5], emb, feat_ex, arc, opt, dropout, nodes, bleu_1, bleu_2, bleu_3, bleu_4, rouge_l,
                          novelty_percentage, fraction_distinct, fraction_within_item_duplicates,
                          vocab_percentage_from_train, nr_duplicates, nr_distinct, within_item_duplicates,
                            vocab_size_predictions, 
                          vocab_size_references, vocab_percentage_inref, vocab_percentage_ref_intrain]
        
        if return_df:
            df = pd.DataFrame([df], columns=['Configuration', 'embedding', 'feature extractor', 'architecture', 'optimizer', 
                                       'dropout', 'Nr. nodes', 'bleu_1', 'bleu_2', 'bleu_3', 'bleu_4', "ROUGE-L",
                                      "Novelty", "Distinctness", "Duplicateness", "Vocabulary", "Number of duplicate captions",
                                      "Number of distinct captions", "Number of within item duplicates", "Vocabulary size predictions",
                                      "Vocabulary size references", "Vocabulary percentage preds / references", "Vocabulary percentage train / references"])
            return df
    
    else:
        return [confname[:-5], bleu_1, bleu_2, bleu_3, bleu_4, rouge_l,
                          novelty_percentage, fraction_distinct, fraction_within_item_duplicates,
                          vocab_percentage_from_train, nr_duplicates, nr_distinct, within_item_duplicates,
                            vocab_size_predictions, 
                          vocab_size_references, vocab_percentage_inref, vocab_percentage_ref_intrain]
    
def get_results(path_res, path_conf, exclude_configs=[]):
    """
    path_res: main path to results folder
    path_conf: main path to config folder
    exclude_configs: list of configuration number to exclude """
    df = []
    if type(path_res) == list:
        paths = path_res
    else:
        paths = [os.path.join(path_res, x) for x in os.listdir(path_res)]
    for path in paths:
        # Skip if config number is excluded
        config_nr = path.split("_")[-1][:-5]
        try:
            if int(config_nr) in exclude_configs:
                continue
        except ValueError:
            print(f"ValueError: {config_nr} is not a number")
            continue
        if path.endswith(".json"):
            if "cropped" in path:
                confname = os.path.basename(path)[21:]
            else:
                confname = os.path.basename(path)[12:]
            with open(path, 'r') as f:
                results = json.load(f)
            try:
                with open(os.path.join(path_conf, confname), 'r') as f2:
                    conf = json.load(f2)
            except FileNotFoundError:
                try: 
                    # For some older files, the config name is slightly different
                    confname = confname[1:]
                    with open(os.path.join(path_conf, confname), 'r') as f2:
                        conf = json.load(f2)
                except FileNotFoundError as e:
                    print(str(e))
                    continue
            try:
                bleu = results['BLEU_SCORES']
                bleu_1 = bleu['BLEU_1']
                bleu_2 = bleu['BLEU_2']
                bleu_3 = bleu['BLEU_3']
                bleu_4 = bleu['BLEU_4']

                # Main novelty measures
                novelty_percentage = results["novelty_score"] 
                fraction_distinct = results["fraction_Distinct"]
                within_item_duplicates = results["within_item_duplicates"]
                fraction_within_item_duplicates = results["fraction_within_duplicates"]
                vocab_percentage_from_train = results["vocab_perc_intrain"]

                # Secondary novelty measures
                nr_duplicates = results["nr_duplicate_total"]
                nr_distinct = results["nr_distinct"]
                vocab_size_predictions = results["vocab_size_predictions"]
                vocab_size_references = results["vocab_size_ref"]
                vocab_percentage_inref = results["vocab_perc_inref"]
                vocab_percentage_ref_intrain = results["vocab_per_test_train"]




                rouge_l = results["ROUGE_SCORES"]["Avg"]["rouge-l"]["f"]

                emb = conf['embedding_name']
                feat_ex = conf['img_folder'].split("_")
                feat_ex = "_".join(feat_ex[:2])
                dropout = conf['dropout']
                opt = conf['optimizer']
                arc = conf['architecture_type']
                nodes = conf['nr_nodes']
            except KeyError as e:
                print(str(e), confname) 

            df.append((confname[:-5], emb, feat_ex, arc, opt, dropout, nodes, bleu_1, bleu_2, bleu_3, bleu_4, rouge_l,
                      novelty_percentage, fraction_distinct, fraction_within_item_duplicates,
                      vocab_percentage_from_train, nr_duplicates, nr_distinct, within_item_duplicates,
                        vocab_size_predictions, 
                      vocab_size_references, vocab_percentage_inref, vocab_percentage_ref_intrain))
    df = pd.DataFrame(df, columns=['Configuration', 'Embedding', 'Feature extractor', 'Architecture', 'Optimizer', 
                                   'Dropout', 'Nr. nodes', 'BLEU-1', 'BLEU-2', 'BLEU-3', 'BLEU-4', "ROUGE-L",
                                  "Novelty", "Distinctness", "Duplicateness", "Vocabulary", "Number of duplicate captions",
                                  "Number of distinct captions", "Number of within item duplicates", "Vocabulary size predictions",
                                  "Vocabulary size references", "Vocabulary percentage preds / references", "Vocabulary percentage train / references"])
    return df

def prepare_for_latex(df):
    replace_dict = {}
    replace_dict["glove_300d_wiki_thr5_emb.txt"] = "GloVe"
    replace_dict["glove_300d_wiki_thr5_emb.p"] = "GloVe"
    replace_dict["fasttext_300d_wiki_thr5_emb.txt"] = "fastText"
    replace_dict["fasttext_300d_wiki_thr5_emb.p"] = "fastText"
    replace_dict["incv3_fromscratch"] = "From scratch"
    replace_dict["incv3_fromFF"] = "From A"
    replace_dict["incv3_noft"] = "A"
    replace_dict["NOT_REALLY_TRUE"] = False
    replace_dict["NOT_REALLY_FALSE"] = True
#     replace_dict[True] = "True"
#     replace_dict[False] = "False"
    # replace values for table
    df.replace(to_replace=replace_dict, inplace=True)
#     df_rl_lat.replace(to_replace={True: "True", False: "False"}, inplace=True)

    
    
    return(df)

Results Ralph Lauren

In [145]:
exclude_configs_rl = list(range(108, 401)) + list(range(450,1000))

In [146]:
df_rl = get_results(path_res_rl, path_conf_rl, exclude_configs_rl)
# Send to excel
df_rl.to_excel(r"C:\Users\s159655\Documents\JADS\Thesis\RESULTS\Results_Paper\RL_RESULTS_ALL_FINAL.xlsx")

file not found..  dict_2_RL_config_1.json
file not found..  dict_RL_config_1.json
file not found..  dict_RL_config_10.json
file not found..  dict_RL_config_11.json
file not found..  dict_RL_config_12.json
file not found..  dict_RL_config_13.json
file not found..  dict_RL_config_14.json
file not found..  dict_RL_config_15.json
file not found..  dict_RL_config_16.json
file not found..  dict_RL_config_17.json
file not found..  dict_RL_config_18.json
file not found..  dict_RL_config_19.json
file not found..  dict_RL_config_2.json
file not found..  dict_RL_config_20.json
file not found..  dict_RL_config_21.json
file not found..  dict_RL_config_22.json
file not found..  dict_RL_config_23.json
file not found..  dict_RL_config_24.json
file not found..  dict_RL_config_3.json
file not found..  dict_RL_config_4.json
file not found..  dict_RL_config_5.json
file not found..  dict_RL_config_6.json
file not found..  dict_RL_config_7.json
file not found..  dict_RL_config_8.json
file not found..  dict_

In [147]:
df_rl.head()

,Configuration,Embedding,Feature extractor,Architecture,Optimizer,Dropout,Nr. nodes,BLEU-1,BLEU-2,BLEU-3,...,Distinctness,Duplicateness,Vocabulary,Number of duplicate captions,Number of distinct captions,Number of within item duplicates,Vocabulary size predictions,Vocabulary size references,Vocabulary percentage preds / references,Vocabulary percentage train / references
0,RL_config_401,glove_300d_wiki_thr5_emb.txt,incv3_fromscratch,parinject,adam,False,256,0.388656,0.322447,0.295829,...,0.522876,0.343783,0.970565,1022,1120,965,1253,1291,0.624315,0.643249
1,RL_config_402,glove_300d_wiki_thr5_emb.txt,incv3_fromFF,parinject,adam,False,256,0.402506,0.332150,0.303942,...,0.489464,0.370146,0.972115,1066,1022,1039,1255,1291,0.625311,0.643249
2,RL_config_403,glove_300d_wiki_thr5_emb.txt,incv3_noft,parinject,adam,False,256,0.406775,0.334619,0.305703,...,0.449123,0.416459,0.932610,1099,896,1169,1204,1291,0.599900,0.643249
3,RL_config_404,fasttext_300d_wiki_thr5_emb.txt,incv3_fromscratch,parinject,adam,False,256,0.370763,0.292733,0.261925,...,0.522084,0.394015,0.892332,963,1052,1106,1152,1291,0.573991,0.643249
4,RL_config_405,fasttext_300d_wiki_thr5_emb.txt,incv3_fromFF,parinject,adam,False,256,0.352065,0.277622,0.248373,...,0.493151,0.386890,0.889233,1036,1008,1086,1148,1291,0.571998,0.643249


In [148]:
# Prepare for latex
df_rl_lat = prepare_for_latex(df_rl)
df_rl_lat = df_rl_lat[["Architecture", "Feature extractor", "Embedding", "Dropout", 
                        "Nr. nodes", "BLEU-1", "BLEU-4", "ROUGE-L", "Novelty", "Distinctness", "Duplicateness", "Vocabulary"]]

# Take average values for exact configuration matches
df_rl_lat = df_rl_lat.groupby(["Architecture", "Feature extractor", "Embedding", "Dropout", "Nr. nodes"]).mean()
df_rl_lat.sort_values(by="BLEU-4", inplace=True, ascending=False)
df_rl_lat = df_rl_lat.round(decimals=3)
df_rl_lat.reset_index(inplace=True)
# df_rl_lat.replace(to_replace={True: "True", False: "False"}, inplace=True)
df_rl_lat.index += 1

In [149]:
df_rl_lat

,Architecture,Feature extractor,Embedding,Dropout,Nr. nodes,BLEU-1,BLEU-4,ROUGE-L,Novelty,Distinctness,Duplicateness,Vocabulary
1,brownlee,From A,fastText,False,256,0.457,0.348,0.380,0.172,0.343,0.433,0.980
2,brownlee,From scratch,fastText,False,256,0.440,0.328,0.362,0.222,0.382,0.422,0.981
3,brownlee,From scratch,GloVe,False,256,0.428,0.321,0.370,0.245,0.414,0.401,0.976
4,brownlee,A,fastText,False,256,0.431,0.315,0.358,0.196,0.365,0.428,0.977
5,brownlee,From A,GloVe,False,256,0.423,0.315,0.356,0.193,0.351,0.409,0.982
6,brownlee,A,GloVe,False,256,0.422,0.308,0.353,0.172,0.326,0.426,0.971
7,parinject,A,GloVe,False,256,0.407,0.290,0.315,0.386,0.449,0.416,0.933
8,parinject,From A,GloVe,False,256,0.403,0.289,0.323,0.373,0.489,0.370,0.972
9,parinject,From scratch,GloVe,False,256,0.389,0.281,0.330,0.394,0.523,0.344,0.971
10,parinject,A,fastText,False,256,0.393,0.274,0.301,0.536,0.546,0.383,0.888


In [150]:
df_rl_lat.to_excel(r"C:\Users\s159655\Documents\JADS\Thesis\RESULTS\Results_Paper\RL_RESULTS_LATEX.xlsx")

In [153]:
df_rl.groupby("Architecture")["BLEU-1"].max()

Architecture
attention    0.126703
brownlee     0.457082
merge        0.369815
parinject    0.406775
Name: BLEU-1, dtype: float64

# For other data sets

In [154]:
exclude_configs = list(range(55, 400)) + list(range(450, 1000))
df_ma = get_results(path_res_ma, path_conf_ma, exclude_configs)
df_ma.to_excel(r"C:\Users\s159655\Documents\JADS\Thesis\RESULTS\Results_Paper\MA_RESULTS_ALL_FINAL.xlsx")
df_ma.groupby("Architecture")["BLEU-1"].max()

Architecture
attention    0.070528
brownlee     0.301591
merge        0.298511
parinject    0.305362
Name: BLEU-1, dtype: float64

In [155]:
df_uo = get_results(path_res_uo, path_conf_uo, exclude_configs)
df_uo.to_excel(r"C:\Users\s159655\Documents\JADS\Thesis\RESULTS\Results_Paper\UO_RESULTS_ALL_FINAL.xlsx")
df_uo.groupby("Architecture")["BLEU-1"].max()

Architecture
attention    0.157803
brownlee     0.317931
merge        0.321261
parinject    0.319152
Name: BLEU-1, dtype: float64

In [156]:
df_ae = get_results(path_res_ae, path_conf_ae, exclude_configs)
df_ae.to_excel(r"C:\Users\s159655\Documents\JADS\Thesis\RESULTS\Results_Paper\AE_RESULTS_ALL_FINAL.xlsx")
df_ae.groupby("Architecture")["BLEU-1"].max()

Architecture
attention    0.104113
brownlee     0.388530
merge        0.391545
parinject    0.398013
Name: BLEU-1, dtype: float64

# Latex output

In [157]:
# Prepare for latex
df_ma_lat = prepare_for_latex(df_ma)
df_ma_lat = df_ma_lat[["Architecture", "Feature extractor", "Embedding", "Dropout", 
                        "Nr. nodes", "BLEU-1", "BLEU-4", "ROUGE-L", "Novelty", "Distinctness", "Duplicateness", "Vocabulary"]]

# Take average values for exact configuration matches
df_ma_lat = df_ma_lat.groupby(["Architecture", "Feature extractor", "Embedding", "Dropout", "Nr. nodes"]).mean()
df_ma_lat.sort_values(by="BLEU-4", inplace=True, ascending=False)
df_ma_lat = df_ma_lat.round(decimals=3)
df_ma_lat.reset_index(inplace=True)
# df_ma_lat.replace(to_replace={True: "True", False: "False"}, inplace=True)
df_ma_lat.index += 1
df_ma_lat.to_excel(r"C:\Users\s159655\Documents\JADS\Thesis\RESULTS\Results_Paper\MA_RESULTS_LATEX.xlsx")

# Prepare for latex
df_uo_lat = prepare_for_latex(df_uo)
df_uo_lat = df_uo_lat[["Architecture", "Feature extractor", "Embedding", "Dropout", 
                        "Nr. nodes", "BLEU-1", "BLEU-4", "ROUGE-L", "Novelty", "Distinctness", "Duplicateness", "Vocabulary"]]

# Take average values for exact configuration matches
df_uo_lat = df_uo_lat.groupby(["Architecture", "Feature extractor", "Embedding", "Dropout", "Nr. nodes"]).mean()
df_uo_lat.sort_values(by="BLEU-4", inplace=True, ascending=False)
df_uo_lat = df_uo_lat.round(decimals=3)
df_uo_lat.reset_index(inplace=True)
# df_uo_lat.replace(to_replace={True: "True", False: "False"}, inplace=True)
df_uo_lat.index += 1
df_uo_lat.to_excel(r"C:\Users\s159655\Documents\JADS\Thesis\RESULTS\Results_Paper\UO_RESULTS_LATEX.xlsx")

# Prepare for latex
df_ae_lat = prepare_for_latex(df_ae)
df_ae_lat = df_ae_lat[["Architecture", "Feature extractor", "Embedding", "Dropout", 
                        "Nr. nodes", "BLEU-1", "BLEU-4", "ROUGE-L", "Novelty", "Distinctness", "Duplicateness", "Vocabulary"]]

# Take average values for exact configuration matches
df_ae_lat = df_ae_lat.groupby(["Architecture", "Feature extractor", "Embedding", "Dropout", "Nr. nodes"]).mean()
df_ae_lat.sort_values(by="BLEU-4", inplace=True, ascending=False)
df_ae_lat = df_ae_lat.round(decimals=3)
df_ae_lat.reset_index(inplace=True)
# df_ae_lat.replace(to_replace={True: "True", False: "False"}, inplace=True)
df_ae_lat.index += 1
df_ae_lat.to_excel(r"C:\Users\s159655\Documents\JADS\Thesis\RESULTS\Results_Paper\AE_RESULTS_LATEX.xlsx")

In [158]:
df_uo_lat

,Architecture,Feature extractor,Embedding,Dropout,Nr. nodes,BLEU-1,BLEU-4,ROUGE-L,Novelty,Distinctness,Duplicateness,Vocabulary
1,brownlee,From A,GloVe,False,256,0.317,0.113,0.267,0.350,0.385,0.394,0.942
2,parinject,From A,fastText,False,256,0.319,0.109,0.275,0.819,0.760,0.282,0.935
3,brownlee,From scratch,GloVe,False,256,0.310,0.109,0.262,0.399,0.482,0.330,0.971
4,merge,From A,fastText,False,256,0.318,0.105,0.265,0.633,0.495,0.389,0.837
5,brownlee,From A,fastText,False,256,0.318,0.105,0.261,0.380,0.391,0.379,0.937
6,brownlee,A,fastText,False,256,0.305,0.103,0.259,0.331,0.379,0.367,0.957
7,brownlee,From scratch,fastText,False,256,0.305,0.103,0.257,0.339,0.467,0.330,0.977
8,brownlee,A,GloVe,False,256,0.303,0.100,0.256,0.373,0.442,0.331,0.963
9,parinject,From A,GloVe,False,256,0.316,0.099,0.269,0.856,0.754,0.274,0.905
10,parinject,From scratch,GloVe,False,256,0.310,0.099,0.261,0.789,0.757,0.275,0.927


In [16]:
df_ae[df_ae["Configuration"].isin([f"AE_config_{i}" for i in range(401, 425)])]

,Configuration,embedding,feature extractor,architecture,optimizer,dropout,Nr. nodes,bleu_1,bleu_2,bleu_3,...,Distinctness,Duplicateness,Vocabulary,Number of duplicate captions,Number of distinct captions,Number of within item duplicates,Vocabulary size predictions,Vocabulary size references,Vocabulary percentage preds / references,Vocabulary percentage train / references
35,AE_config_401,glove_300d_wiki_thr5_emb.p,incv3_fromscratch,parinject,adam,False,256,0.346844,0.307483,0.290268,...,0.295377,0.542659,0.936312,1646,690,2118,985,1052,0.585612,0.625446
36,AE_config_402,glove_300d_wiki_thr5_emb.p,incv3_fromFF,parinject,adam,False,256,0.398013,0.361703,0.344602,...,0.333812,0.612862,0.932510,1393,698,2392,981,1052,0.583234,0.625446
37,AE_config_403,glove_300d_wiki_thr5_emb.p,incv3_noft,parinject,adam,False,256,0.354868,0.314425,0.296917,...,0.312446,0.549065,0.927757,1602,728,2143,976,1052,0.580262,0.625446
38,AE_config_404,fasttext_300d_wiki_thr5_emb.p,incv3_fromscratch,parinject,adam,False,256,0.355618,0.317455,0.300904,...,0.352268,0.556751,0.897338,1471,800,2173,944,1052,0.561237,0.625446
39,AE_config_405,fasttext_300d_wiki_thr5_emb.p,incv3_fromFF,parinject,adam,False,256,0.369988,0.333715,0.317342,...,0.336488,0.609019,0.928707,1402,711,2377,977,1052,0.580856,0.625446
40,AE_config_406,fasttext_300d_wiki_thr5_emb.p,incv3_noft,parinject,adam,False,256,0.340585,0.301612,0.284287,...,0.376203,0.558288,0.927757,1426,860,2179,976,1052,0.580262,0.625446
41,AE_config_407,glove_300d_wiki_thr5_emb.p,incv3_fromscratch,brownlee,adam,False,256,0.347331,0.307588,0.289636,...,0.225959,0.548552,0.945817,1795,524,2141,995,1052,0.591558,0.625446
42,AE_config_408,glove_300d_wiki_thr5_emb.p,incv3_fromFF,brownlee,adam,False,256,0.388530,0.349603,0.331620,...,0.200000,0.653856,0.897338,1592,398,2552,944,1052,0.561237,0.625446
43,AE_config_409,glove_300d_wiki_thr5_emb.p,incv3_noft,brownlee,adam,False,256,0.355736,0.318150,0.301380,...,0.235112,0.564950,0.945817,1734,533,2205,995,1052,0.591558,0.625446
45,AE_config_410,fasttext_300d_wiki_thr5_emb.p,incv3_fromscratch,brownlee,adam,False,256,0.363777,0.326020,0.308934,...,0.268041,0.546759,0.950570,1704,624,2134,1000,1052,0.594530,0.625446


In [26]:
df_ae[df_ae["architecture"]=="brownlee"].sort_values(by="bleu_4", ascending=False).head(10)

,Configuration,embedding,feature extractor,architecture,optimizer,dropout,Nr. nodes,bleu_1,bleu_2,bleu_3,...,Distinctness,Duplicateness,Vocabulary,Number of duplicate captions,Number of distinct captions,Number of within item duplicates,Vocabulary size predictions,Vocabulary size references,Vocabulary percentage preds / references,Vocabulary percentage train / references
42,AE_config_408,glove_300d_wiki_thr5_emb.p,incv3_fromFF,brownlee,adam,False,256,0.388530,0.349603,0.331620,...,0.200000,0.653856,0.897338,1592,398,2552,944,1052,0.561237,0.625446
46,AE_config_411,fasttext_300d_wiki_thr5_emb.p,incv3_fromFF,brownlee,adam,False,256,0.385300,0.348324,0.330603,...,0.288435,0.583397,0.953422,1569,636,2277,1003,1052,0.596314,0.625446
45,AE_config_410,fasttext_300d_wiki_thr5_emb.p,incv3_fromscratch,brownlee,adam,False,256,0.363777,0.326020,0.308934,...,0.268041,0.546759,0.950570,1704,624,2134,1000,1052,0.594530,0.625446
47,AE_config_412,fasttext_300d_wiki_thr5_emb.p,incv3_noft,brownlee,adam,False,256,0.362040,0.323564,0.305794,...,0.248834,0.535998,0.972433,1772,587,2092,1023,1052,0.608205,0.625446
43,AE_config_409,glove_300d_wiki_thr5_emb.p,incv3_noft,brownlee,adam,False,256,0.355736,0.318150,0.301380,...,0.235112,0.564950,0.945817,1734,533,2205,995,1052,0.591558,0.625446
41,AE_config_407,glove_300d_wiki_thr5_emb.p,incv3_fromscratch,brownlee,adam,False,256,0.347331,0.307588,0.289636,...,0.225959,0.548552,0.945817,1795,524,2141,995,1052,0.591558,0.625446
63,AE_config_45,fasttext_300d_wiki_thr5_emb.p,incv3_fromFF,brownlee,adam,NOT_REALLY_FALSE,256,0.172858,0.137003,0.123621,...,0.002451,0.941071,0.103612,814,2,3673,109,1052,0.064804,0.625446
32,AE_config_39,glove_300d_wiki_thr5_emb.p,incv3_fromFF,brownlee,adam,NOT_REALLY_FALSE,256,0.178229,0.134861,0.118285,...,0.007073,0.873174,0.081749,1123,8,3408,86,1052,0.051130,0.625446
62,AE_config_44,fasttext_300d_wiki_thr5_emb.p,incv3_fromscratch,brownlee,adam,True,256,0.162850,0.126477,0.113629,...,0.001135,0.934153,0.074144,880,1,3646,78,1052,0.046373,0.625446
31,AE_config_38,glove_300d_wiki_thr5_emb.p,incv3_fromscratch,brownlee,adam,True,256,0.163848,0.121878,0.106167,...,0.004525,0.925698,0.074144,880,4,3613,78,1052,0.046373,0.625446


In [27]:
df_rl[df_rl["architecture"]=="brownlee"].sort_values(by="bleu_4", ascending=False).head(10)

,Configuration,embedding,feature extractor,architecture,optimizer,dropout,Nr. nodes,bleu_1,bleu_2,bleu_3,...,Distinctness,Duplicateness,Vocabulary,Number of duplicate captions,Number of distinct captions,Number of within item duplicates,Vocabulary size predictions,Vocabulary size references,Vocabulary percentage preds / references,Vocabulary percentage train / references
10,RL_config_411,fasttext_300d_wiki_thr5_emb.txt,incv3_fromFF,brownlee,adam,False,256,0.457082,0.391399,0.363797,...,0.343496,0.432846,0.979861,1292,676,1215,1265,1291,0.630294,0.643249
9,RL_config_410,fasttext_300d_wiki_thr5_emb.txt,incv3_fromscratch,brownlee,adam,False,256,0.440020,0.372032,0.343705,...,0.382265,0.422159,0.980635,1233,763,1185,1266,1291,0.630792,0.643249
6,RL_config_407,glove_300d_wiki_thr5_emb.txt,incv3_fromscratch,brownlee,adam,False,256,0.428402,0.362934,0.335903,...,0.413504,0.400784,0.975988,1190,839,1125,1260,1291,0.627803,0.643249
11,RL_config_412,fasttext_300d_wiki_thr5_emb.txt,incv3_noft,brownlee,adam,False,256,0.430984,0.360806,0.331661,...,0.364599,0.427859,0.976762,1260,723,1201,1261,1291,0.628301,0.643249
7,RL_config_408,glove_300d_wiki_thr5_emb.txt,incv3_fromFF,brownlee,adam,False,256,0.423310,0.357110,0.330062,...,0.351473,0.408978,0.982184,1299,704,1148,1268,1291,0.631789,0.643249
8,RL_config_409,glove_300d_wiki_thr5_emb.txt,incv3_noft,brownlee,adam,False,256,0.421932,0.352089,0.323712,...,0.326284,0.425721,0.971340,1338,648,1195,1254,1291,0.624813,0.643249
67,RL_config_96,fasttext_300d_wiki_thr5_emb.txt,incv3_noft,brownlee,adam,True,256,0.127293,0.067726,0.050747,...,0.045576,0.886356,0.090627,712,34,2488,117,1291,0.058296,0.643249
56,RL_config_85,glove_300d_wiki_thr5_emb.txt,incv3_fromscratch,brownlee,adam,NOT_REALLY_FALSE,256,0.156002,0.071239,0.048050,...,0.225764,0.863199,0.086754,583,170,2423,112,1291,0.055805,0.643249
66,RL_config_95,fasttext_300d_wiki_thr5_emb.txt,incv3_noft,brownlee,adam,NOT_REALLY_FALSE,256,0.098576,0.047686,0.034453,...,0.013720,0.917349,0.100697,647,9,2575,130,1291,0.064773,0.643249
57,RL_config_86,glove_300d_wiki_thr5_emb.txt,incv3_fromscratch,brownlee,adam,True,256,0.127094,0.055434,0.036577,...,0.004702,0.927681,0.094500,635,3,2604,122,1291,0.060787,0.643249


# Robustness Check

In [8]:
exclude_configs_ro = list(range(1, 501)) + list(range(510, 1000))
ro_rl = get_results(path_res_rl, path_conf_rl, exclude_configs_ro)

In [9]:
ro_rl

,Configuration,embedding,feature extractor,architecture,optimizer,dropout,Nr. nodes,bleu_1,bleu_2,bleu_3,...,Distinctness,Duplicateness,Vocabulary,Number of duplicate captions,Number of distinct captions,Number of within item duplicates,Vocabulary size predictions,Vocabulary size references,Vocabulary percentage preds / references,Vocabulary percentage train / references
0,RL_config_501,glove_300d_wiki_thr5_emb.txt,incv3_fromFF,merge,adam,False,256,0.378919,0.301122,0.271886,...,0.002058,0.988956,0.265686,485,1,2776,343,1291,0.170902,0.643249
1,RL_config_502,glove_300d_wiki_thr5_emb.txt,incv3_fromFF,parinject,adam,False,256,0.398363,0.319370,0.288492,...,0.002058,0.988956,0.267235,485,1,2776,345,1291,0.171898,0.643249
2,RL_config_503,fasttext_300d_wiki_thr5_emb.txt,incv3_fromFF,brownlee,adam,False,256,0.380410,0.305836,0.277058,...,0.002058,0.988956,0.250968,485,1,2776,324,1291,0.161435,0.643249
3,RL_config_504,glove_300d_wiki_thr5_emb.txt,incv3_fromFF,merge,adam,False,256,0.354175,0.283156,0.257398,...,0.000000,0.988956,0.217661,486,0,2776,281,1291,0.140010,0.643249
4,RL_config_505,glove_300d_wiki_thr5_emb.txt,incv3_fromFF,parinject,adam,False,256,0.383022,0.305450,0.276031,...,0.002058,0.988956,0.252517,485,1,2776,326,1291,0.162431,0.643249
5,RL_config_506,fasttext_300d_wiki_thr5_emb.txt,incv3_fromFF,brownlee,adam,False,256,0.389483,0.313784,0.284660,...,0.002058,0.988956,0.288923,485,1,2776,373,1291,0.185850,0.643249


In [55]:
# Get results for respective configurations;  6 (66 for RL), 402 and 411
with open(os.path.join(path_res_rl, "results_dict_RL_config_66.json"), 'r') as f:
    rl_merge = json.load(f)
with open(os.path.join(path_res_rl, "results_dictRL_config_501.json"), 'r') as f:
    rl_merge_rob = json.load(f)
with open(os.path.join(path_res_rl, "results_dictRL_config_402.json"), 'r') as f:
    rl_par = json.load(f)
with open(os.path.join(path_res_rl, "results_dictRL_config_502.json"), 'r') as f:
    rl_par_rob = json.load(f)
with open(os.path.join(path_res_rl, "results_dictRL_config_411.json"), 'r') as f:
    rl_brown = json.load(f)
with open(os.path.join(path_res_rl, "results_dictRL_config_506.json"), 'r') as f:
    rl_brown_rob = json.load(f)

In [56]:
results = []
results.append(("RL_MERGE", rl_merge["BLEU_SCORES"], rl_merge["ROUGE_SCORES"]["Avg"]["rouge-l"]["f"], 
                rl_merge["novelty_score"], rl_merge["fraction_Distinct"], rl_merge["fraction_within_duplicates"], 
                rl_merge_rob["BLEU_SCORES"], rl_merge_rob["ROUGE_SCORES"]["Avg"]["rouge-l"]["f"], 
                rl_merge_rob["novelty_score"], rl_merge_rob["fraction_Distinct"], rl_merge_rob["fraction_within_duplicates"]))
results.append(("RL_PAR", rl_par["BLEU_SCORES"], rl_par["ROUGE_SCORES"]["Avg"]["rouge-l"]["f"], 
                rl_par["novelty_score"], rl_par["fraction_Distinct"], rl_par["fraction_within_duplicates"], 
                rl_par_rob["BLEU_SCORES"], rl_par_rob["ROUGE_SCORES"]["Avg"]["rouge-l"]["f"], 
                rl_par_rob["novelty_score"], rl_par_rob["fraction_Distinct"], rl_par_rob["fraction_within_duplicates"]))

results.append(("RL_BROWN", rl_brown["BLEU_SCORES"], rl_brown["ROUGE_SCORES"]["Avg"]["rouge-l"]["f"], 
                rl_brown["novelty_score"], rl_brown["fraction_Distinct"], rl_brown["fraction_within_duplicates"], 
                rl_brown_rob["BLEU_SCORES"], rl_brown_rob["ROUGE_SCORES"]["Avg"]["rouge-l"]["f"], 
                rl_brown_rob["novelty_score"], rl_brown_rob["fraction_Distinct"], rl_brown_rob["fraction_within_duplicates"]))

In [57]:
ro_uo = get_results(path_res_uo, path_conf_uo, exclude_configs_ro)
ro_uo

,Configuration,embedding,feature extractor,architecture,optimizer,dropout,Nr. nodes,bleu_1,bleu_2,bleu_3,...,Distinctness,Duplicateness,Vocabulary,Number of duplicate captions,Number of distinct captions,Number of within item duplicates,Vocabulary size predictions,Vocabulary size references,Vocabulary percentage preds / references,Vocabulary percentage train / references
0,UO_config_501,glove_300d_wiki_thr5_emb.txt,incv3_fromFF,merge,adam,False,256,0.291466,0.156202,0.099290,...,0.000873,0.999514,0.177703,1144,1,6173,424,2386,0.106133,0.597247
1,UO_config_502,glove_300d_wiki_thr5_emb.txt,incv3_fromFF,parinject,adam,False,256,0.297707,0.169129,0.110258,...,0.000873,0.999514,0.186505,1144,1,6173,445,2386,0.111389,0.597247
2,UO_config_503,fasttext_300d_wiki_thr5_emb.txt,incv3_fromFF,brownlee,adam,False,256,0.281355,0.165605,0.108800,...,0.000873,0.999514,0.129086,1144,1,6173,308,2386,0.077096,0.597247
3,UO_config_504,glove_300d_wiki_thr5_emb.txt,incv3_fromFF,merge,adam,False,256,0.292158,0.164923,0.109761,...,0.000873,0.999514,0.190696,1144,1,6173,455,2386,0.113892,0.597247
4,UO_config_505,glove_300d_wiki_thr5_emb.txt,incv3_fromFF,parinject,adam,False,256,0.289740,0.162416,0.105379,...,0.000873,0.999514,0.177703,1144,1,6173,424,2386,0.106133,0.597247
5,UO_config_506,fasttext_300d_wiki_thr5_emb.txt,incv3_fromFF,brownlee,adam,False,256,0.280247,0.166488,0.109285,...,0.000873,0.999514,0.131182,1144,1,6173,313,2386,0.078348,0.597247


In [58]:
ro_ae = get_results(path_res_ae, path_conf_ae, exclude_configs_ro)
ro_ae

,Configuration,embedding,feature extractor,architecture,optimizer,dropout,Nr. nodes,bleu_1,bleu_2,bleu_3,...,Distinctness,Duplicateness,Vocabulary,Number of duplicate captions,Number of distinct captions,Number of within item duplicates,Vocabulary size predictions,Vocabulary size references,Vocabulary percentage preds / references,Vocabulary percentage train / references
0,AE_config_501,glove_300d_wiki_thr5_emb.p,incv3_fromFF,merge,adam,False,256,0.230457,0.185407,0.164981,...,0.00161,0.987702,0.183460,620,1,3855,193,1052,0.114744,0.625446
1,AE_config_502,glove_300d_wiki_thr5_emb.p,incv3_fromFF,parinject,adam,False,256,0.225866,0.173665,0.152009,...,0.00161,0.987702,0.166350,620,1,3855,175,1052,0.104043,0.625446
2,AE_config_503,fasttext_300d_wiki_thr5_emb.p,incv3_fromFF,brownlee,adam,False,256,0.230132,0.185211,0.165171,...,0.00161,0.987702,0.167300,620,1,3855,176,1052,0.104637,0.625446
3,AE_config_504,glove_300d_wiki_thr5_emb.p,incv3_fromFF,merge,adam,False,256,0.220863,0.170218,0.149299,...,0.00161,0.987702,0.175856,620,1,3855,185,1052,0.109988,0.625446
4,AE_config_505,glove_300d_wiki_thr5_emb.p,incv3_fromFF,parinject,adam,False,256,0.229112,0.177965,0.157093,...,0.00161,0.987702,0.162548,620,1,3855,171,1052,0.101665,0.625446
5,AE_config_506,fasttext_300d_wiki_thr5_emb.p,incv3_fromFF,brownlee,adam,False,256,0.227854,0.181672,0.161403,...,0.00161,0.987702,0.162548,620,1,3855,171,1052,0.101665,0.625446


In [59]:
ro_ma = get_results(path_res_ma, path_conf_ma, exclude_configs_ro)
ro_ma

,Configuration,embedding,feature extractor,architecture,optimizer,dropout,Nr. nodes,bleu_1,bleu_2,bleu_3,...,Distinctness,Duplicateness,Vocabulary,Number of duplicate captions,Number of distinct captions,Number of within item duplicates,Vocabulary size predictions,Vocabulary size references,Vocabulary percentage preds / references,Vocabulary percentage train / references
0,MA_config_501,glove_300d_wiki_thr5_emb.p,incv3_fromFF,merge,adam,False,256,0.292589,0.204598,0.171054,...,0.005908,0.973294,0.281141,673,4,2624,562,1999,0.185970,0.661482
1,MA_config_502,glove_300d_wiki_thr5_emb.p,incv3_fromFF,parinject,adam,False,256,0.282532,0.204942,0.174343,...,0.005908,0.973294,0.219610,673,4,2624,439,1999,0.145268,0.661482
2,MA_config_503,fasttext_300d_wiki_thr5_emb.p,incv3_fromFF,brownlee,adam,False,256,0.294848,0.211125,0.179614,...,0.005908,0.973294,0.311656,673,4,2624,623,1999,0.206155,0.661482
3,MA_config_504,glove_300d_wiki_thr5_emb.p,incv3_fromFF,merge,adam,False,256,0.321692,0.238958,0.207430,...,0.005908,0.973294,0.322161,673,4,2624,644,1999,0.213104,0.661482
4,MA_config_505,glove_300d_wiki_thr5_emb.p,incv3_fromFF,parinject,adam,False,256,0.287992,0.203527,0.170693,...,0.005908,0.973294,0.236118,673,4,2624,472,1999,0.156188,0.661482
5,MA_config_506,fasttext_300d_wiki_thr5_emb.p,incv3_fromFF,brownlee,adam,False,256,0.308439,0.226733,0.194231,...,0.005908,0.973294,0.299150,673,4,2624,598,1999,0.197882,0.661482


In [20]:
# Get results for respective configurations;  6 , 402 and 411
results = []
for w in ["UO", "AE", "MA", "RL"]:
    if w == "UO":
        pa = path_res_uo
    if w == "AE":
        pa = path_res_ae 
    if w == "MA":
        pa = path_res_ma
    if w == "RL":
        pa = path_res_rl
        with open(os.path.join(pa, f"results_dict_{w}_config_66.json"), 'r') as f:
            merge = json.load(f)
    else:
        with open(os.path.join(pa, f"results_dict{w}_config_6.json"), 'r') as f:
            merge = json.load(f)
    with open(os.path.join(pa, f"results_dict{w}_config_501.json"), 'r') as f:
        merge_rob = json.load(f)
    with open(os.path.join(pa, f"results_dict{w}_config_402.json"), 'r') as f:
        par = json.load(f)
    with open(os.path.join(pa, f"results_dict{w}_config_502.json"), 'r') as f:
        par_rob = json.load(f)
    with open(os.path.join(pa, f"results_dict{w}_config_411.json"), 'r') as f:
        brown = json.load(f)
    with open(os.path.join(pa, f"results_dict{w}_config_506.json"), 'r') as f:
        brown_rob = json.load(f)
            # load image_cat combined models
    with open(os.path.join(pa, f"results_dict{w}_config_701.json"), 'r') as f:
        merge_imgcat = json.load(f)
    with open(os.path.join(pa, f"results_dict{w}_config_702.json"), 'r') as f:
        par_imgcat = json.load(f)
    with open(os.path.join(pa, f"results_dict{w}_config_703.json"), 'r') as f:
        brown_imgcat = json.load(f)
    print(w)    
    results.append((f"{w} merge - img", merge["BLEU_SCORES"]["BLEU_1"], merge["BLEU_SCORES"]["BLEU_2"], 
                    merge["BLEU_SCORES"]["BLEU_3"], merge["BLEU_SCORES"]["BLEU_4"], 
                    merge["ROUGE_SCORES"]["Avg"]["rouge-l"]["f"], 
                merge["novelty_score"], merge["fraction_Distinct"], merge["fraction_within_duplicates"], 
                    merge["vocab_perc_intrain"]))
    results.append((f"{w} merge - cat",
                merge_rob["BLEU_SCORES"]["BLEU_1"], merge_rob["BLEU_SCORES"]["BLEU_2"], 
                    merge_rob["BLEU_SCORES"]["BLEU_3"], merge_rob["BLEU_SCORES"]["BLEU_4"],  merge_rob["ROUGE_SCORES"]["Avg"]["rouge-l"]["f"], 
                merge_rob["novelty_score"], merge_rob["fraction_Distinct"], merge_rob["fraction_within_duplicates"], merge_rob["vocab_perc_intrain"]))
    results.append((f"{w} merge - img \& cat", merge_imgcat["BLEU_SCORES"]["BLEU_1"], merge_imgcat["BLEU_SCORES"]["BLEU_2"], 
                merge_imgcat["BLEU_SCORES"]["BLEU_3"], merge_imgcat["BLEU_SCORES"]["BLEU_4"], 
                merge_imgcat["ROUGE_SCORES"]["Avg"]["rouge-l"]["f"], 
            merge_imgcat["novelty_score"], merge_imgcat["fraction_Distinct"], merge_imgcat["fraction_within_duplicates"], 
                merge_imgcat["vocab_perc_intrain"]))
                                                                          
    results.append((f"{w} parinject - img", par["BLEU_SCORES"]["BLEU_1"], par["BLEU_SCORES"]["BLEU_2"], 
                    par["BLEU_SCORES"]["BLEU_3"], par["BLEU_SCORES"]["BLEU_4"],
                    par["ROUGE_SCORES"]["Avg"]["rouge-l"]["f"], 
                    par["novelty_score"], par["fraction_Distinct"], par["fraction_within_duplicates"], par["vocab_perc_intrain"]))
    
    results.append((f"{w} parinject - cat", par_rob["BLEU_SCORES"]["BLEU_1"], par_rob["BLEU_SCORES"]["BLEU_2"], 
                    par_rob["BLEU_SCORES"]["BLEU_3"], par_rob["BLEU_SCORES"]["BLEU_4"], par_rob["ROUGE_SCORES"]["Avg"]["rouge-l"]["f"], 
                    par_rob["novelty_score"], par_rob["fraction_Distinct"], par_rob["fraction_within_duplicates"], merge_imgcat["vocab_perc_intrain"]))
    
    results.append((f"{w} parinject - img \& cat", par_imgcat["BLEU_SCORES"]["BLEU_1"], par_imgcat["BLEU_SCORES"]["BLEU_2"], 
                    par_imgcat["BLEU_SCORES"]["BLEU_3"], par_imgcat["BLEU_SCORES"]["BLEU_4"],
                    par_imgcat["ROUGE_SCORES"]["Avg"]["rouge-l"]["f"], 
                    par_imgcat["novelty_score"], par_imgcat["fraction_Distinct"], par_imgcat["fraction_within_duplicates"], 
                    par_imgcat["vocab_perc_intrain"]))
    
    results.append((f"{w} brownlee - img", brown["BLEU_SCORES"]["BLEU_1"], brown["BLEU_SCORES"]["BLEU_2"], 
                    brown["BLEU_SCORES"]["BLEU_3"], brown["BLEU_SCORES"]["BLEU_4"], brown["ROUGE_SCORES"]["Avg"]["rouge-l"]["f"], 
                    brown["novelty_score"], brown["fraction_Distinct"], brown["fraction_within_duplicates"], brown["vocab_perc_intrain"]))
    
    
    results.append((f"{w} brownlee - cat", brown_rob["BLEU_SCORES"]["BLEU_1"], brown_rob["BLEU_SCORES"]["BLEU_2"], 
                    brown_rob["BLEU_SCORES"]["BLEU_3"], brown_rob["BLEU_SCORES"]["BLEU_4"], brown_rob["ROUGE_SCORES"]["Avg"]["rouge-l"]["f"], 
                    brown_rob["novelty_score"], brown_rob["fraction_Distinct"], brown_rob["fraction_within_duplicates"], brown_rob["vocab_perc_intrain"]))
    
    results.append((f"{w} brownlee - img \& cat", brown_imgcat["BLEU_SCORES"]["BLEU_1"], brown_imgcat["BLEU_SCORES"]["BLEU_2"], 
                    brown_imgcat["BLEU_SCORES"]["BLEU_3"], brown_imgcat["BLEU_SCORES"]["BLEU_4"], brown_imgcat["ROUGE_SCORES"]["Avg"]["rouge-l"]["f"], 
                    brown_imgcat["novelty_score"], brown_imgcat["fraction_Distinct"], brown_imgcat["fraction_within_duplicates"],
                    brown_imgcat["vocab_perc_intrain"]))

UO
AE
MA
RL


In [25]:
results_ro = pd.DataFrame(results, columns=["Architecture", "BLEU_1", "BLEU_2", "BLEU_3", "BLEU_4", "ROUGE-L", "Novelty", "Distinctness", "Duplicateness", "Vocab"])

In [26]:
results_ro.head()

,Architecture,BLEU_1,BLEU_2,BLEU_3,BLEU_4,ROUGE-L,Novelty,Distinctness,Duplicateness,Vocab
0,UO merge - img,0.312579,0.188071,0.129381,0.097371,0.259068,0.618038,0.471627,0.423575,0.837804
1,UO merge - cat,0.291466,0.156202,0.099290,0.069370,0.231011,0.622409,0.000873,0.999514,0.177703
2,UO merge - img \& cat,0.327152,0.207302,0.149958,0.118552,0.276353,0.313957,0.349021,0.502591,0.922883
3,UO parinject - img,0.315605,0.191763,0.132281,0.099004,0.268908,0.856056,0.754383,0.273802,0.904862
4,UO parinject - cat,0.297707,0.169129,0.110258,0.078556,0.247077,0.629534,0.000873,0.999514,0.922883


In [27]:
results_ro.to_excel(r"C:\Users\s159655\Documents\JADS\Thesis\RESULTS\Results_Paper\Robustness_results.xlsx")

In [40]:
import pickle
preds = rl_merge["PREDICTIONS"]
with open(r"C:\Users\s159655\Documents\JADS\Thesis\RESULTS\Results_Paper\train_descs_RALPH_LAUREN.p", "rb") as f:
    train_descs = pickle.load(f)

In [63]:
with open(os.path.join(path_res_ma, f"results_dict{w}_config_502.json"), 'r') as f:
        par_rob = json.load(f)

## Robustness: Combined results

In [47]:
ro_res_combined = []
rr_path = r"C:\Users\s159655\Documents\JADS\Thesis\RESULTS\Results_Paper\ROBUSTNESS"
rc_path = r"C:\Users\s159655\Documents\JADS\Thesis\Code\Train_Test\Configurations\RL_configs"
r_m = get_result_config(os.path.join(rr_path, "COMBINED_config_501.json"), os.path.join(rc_path, "RL_config_501.json"), False, False)
r_p = get_result_config(os.path.join(rr_path, "COMBINED_config_502.json"), os.path.join(rc_path, "RL_config_502.json"), False, False)
r_b = get_result_config(os.path.join(rr_path, "COMBINED_config_506.json"), os.path.join(rc_path, "RL_config_506.json"), False, False)

r_m_i = get_result_config(os.path.join(rr_path, "COMBINED_config_6.json"), os.path.join(rc_path, "RL_config_66.json"), False, False)
r_p_i = get_result_config(os.path.join(rr_path, "COMBINED_config_402.json"), os.path.join(rc_path, "RL_config_402.json"), False, False)
r_b_i = get_result_config(os.path.join(rr_path, "COMBINED_config_411.json"), os.path.join(rc_path, "RL_config_411.json"), False, False)

ro_res_combined.extend([r_m, r_m_i, r_p, r_p_i, r_b, r_b_i])

In [180]:
ro_df = pd.DataFrame(ro_res_combined, columns=['Configuration', 'bleu_1', 'bleu_2', 'bleu_3', 'bleu_4', "ROUGE-L",
                                      "Novelty", "Distinctness", "Duplicateness", "Vocabulary", "Number of duplicate captions",
                                      "Number of distinct captions", "Number of within item duplicates", "Vocabulary size predictions",
                                      "Vocabulary size references", "Vocabulary percentage preds / references", "Vocabulary percentage train / references"])

In [181]:
ro_df = ro_df.round(3)
ro_df

,Configuration,bleu_1,bleu_2,bleu_3,bleu_4,ROUGE-L,Novelty,Distinctness,Duplicateness,Vocabulary,Number of duplicate captions,Number of distinct captions,Number of within item duplicates,Vocabulary size predictions,Vocabulary size references,Vocabulary percentage preds / references,Vocabulary percentage train / references
0,RL_config_501,0.304,0.203,0.163,0.141,0.237,0.383,0.002,0.990,0.272,2922,7,15428,1119,4121,0.161,0.593
1,RL_config_66,0.331,0.236,0.195,0.172,0.285,0.333,0.295,0.502,0.856,7225,3027,7817,3528,4121,0.508,0.593
2,RL_config_502,0.313,0.213,0.170,0.147,0.246,0.475,0.002,0.990,0.255,2922,7,15428,1051,4121,0.151,0.593
3,RL_config_402,0.339,0.248,0.207,0.184,0.308,0.556,0.601,0.357,0.920,4713,7087,5559,3791,4121,0.546,0.593
4,RL_config_506,0.306,0.215,0.175,0.152,0.253,0.453,0.002,0.990,0.263,2922,7,15428,1084,4121,0.156,0.593
5,RL_config_411,0.349,0.259,0.219,0.196,0.310,0.253,0.357,0.412,0.956,7269,4044,6425,3940,4121,0.567,0.593


In [182]:
ro_df.to_excel(r"C:\Users\s159655\Documents\JADS\Thesis\RESULTS\Results_Paper\ROBUSTNESS\Robustness_combined.xlsx")

# Multiple web shop models comparison

In [184]:
mw_results = []
mw_p_res = r"C:\Users\s159655\Documents\JADS\Thesis\RESULTS\Results_Paper\COMBINED"
mw_p_conf = r"C:\Users\s159655\Documents\JADS\Thesis\Code\Train_Test\Configurations\co_configs"
for i in range(21, 29):
    config_name = f"combined_config_{i}.json"
    print(config_name)
    pr = os.path.join(mw_p_res, f"results_dict_{config_name}")
    pc = os.path.join(mw_p_conf, config_name)
    
    results = get_result_config(pr, pc, False, False)
    with open(pc, 'r') as f:
        config = json.load(f)
    # add relevant config info
    concat_add = config["concat_add"]
    concat_moment = config["concat_moment"]
    webshop_embedding = config["webshop_embedding"]
    results = [results[0]] + [concat_add, concat_moment, webshop_embedding] + results[1:]
    
    mw_results.append(results)
    
# Add individual models info
i_p_res = r"C:\Users\s159655\Documents\JADS\Thesis\RESULTS\Results_Paper\MULTIPLE_WEBSHOPS\COMBINED_config_10.json"
i_p_conf = os.path.join(mw_p_conf, config_name)
res_ind = get_result_config(i_p_res, i_p_conf, False, False)
res_ind = ["Combination of Individual Models"] + ["-", "-", "-"] + res_ind[1:]

mw_results.append(res_ind)

mw_df = pd.DataFrame(mw_results, columns=['Configuration', 'concat_add', 'combine moment', 'webshop embedding', 'bleu_1', 'bleu_2', 'bleu_3', 'bleu_4', "ROUGE-L",
                                      "Novelty", "Distinctness", "Duplicateness", "Vocabulary", "Number of duplicate captions",
                                      "Number of distinct captions", "Number of within item duplicates", "Vocabulary size predictions",
                                      "Vocabulary size references", "Vocabulary percentage preds / references", "Vocabulary percentage train / references"])

combined_config_21.json
combined_config_22.json
combined_config_23.json
combined_config_24.json
combined_config_25.json
combined_config_26.json
combined_config_27.json
combined_config_28.json


In [185]:
mw_df

,Configuration,concat_add,combine moment,webshop embedding,bleu_1,bleu_2,bleu_3,bleu_4,ROUGE-L,Novelty,Distinctness,Duplicateness,Vocabulary,Number of duplicate captions,Number of distinct captions,Number of within item duplicates,Vocabulary size predictions,Vocabulary size references,Vocabulary percentage preds / references,Vocabulary percentage train / references
0,combined_config_21,concatenate,early,False,0.280926,0.183668,0.143608,0.122409,0.228774,0.628995,0.341702,0.449878,0.803688,7080,3675,7010,3312,4121,0.476890,0.593377
1,combined_config_22,concatenate,early,True,0.274327,0.180805,0.142628,0.122438,0.229539,0.592414,0.313893,0.471121,0.820675,7235,3310,7341,3382,4121,0.486969,0.593377
2,combined_config_23,concatenate,late,False,0.279664,0.182816,0.143296,0.122677,0.228640,0.591387,0.320544,0.460210,0.854162,7296,3442,7171,3520,4121,0.506839,0.593377
3,combined_config_24,concatenate,late,True,0.278598,0.182745,0.143880,0.123645,0.223503,0.552946,0.313937,0.462713,0.842029,7310,3345,7210,3470,4121,0.499640,0.593377
4,combined_config_25,add,early,False,0.267562,0.172310,0.133705,0.113672,0.217317,0.567835,0.307022,0.493903,0.774569,6997,3100,7696,3192,4121,0.459611,0.593377
5,combined_config_26,add,early,True,0.269818,0.171851,0.131571,0.110511,0.216557,0.599666,0.294588,0.470479,0.738170,7404,3092,7331,3042,4121,0.438013,0.593377
6,combined_config_27,add,late,False,0.262990,0.166345,0.128588,0.108920,0.202737,0.634643,0.377989,0.444616,0.735744,6686,4063,6928,3032,4121,0.436573,0.593377
7,combined_config_28,add,late,True,0.270219,0.172282,0.133501,0.113459,0.207502,0.597484,0.331752,0.464446,0.739626,7044,3497,7237,3048,4121,0.438877,0.593377
8,Combination of Individual Models,-,-,-,0.304126,0.212109,0.173698,0.153047,0.261448,0.330895,0.243404,0.505070,0.817277,7771,2500,7870,3368,4121,0.484953,0.593377


In [187]:
mw_df = mw_df.round(3)
mw_df.to_excel(r"C:\Users\s159655\Documents\JADS\Thesis\RESULTS\Results_Paper\MULTIPLE_WEBSHOSP\Multiple_Webshops.xlsx")

# Further improving results

In [68]:
tr_res_combined = []
fr_path = r"C:\Users\s159655\Documents\JADS\Thesis\RESULTS\Results_Paper\IMPROVING RESULTS"
rc_path = r"C:\Users\s159655\Documents\JADS\Thesis\Code\Train_Test\Configurations\RL_configs"
r_m = get_result_config(os.path.join(fr_path, "COMBINED_config_601.json"), os.path.join(rc_path, "RL_config_601.json"), False, False)
r_p = get_result_config(os.path.join(fr_path, "COMBINED_config_602.json"), os.path.join(rc_path, "RL_config_602.json"), False, False)
r_b = get_result_config(os.path.join(fr_path, "COMBINED_config_603.json"), os.path.join(rc_path, "RL_config_603.json"), False, False)

r_m_i = get_result_config(os.path.join(rr_path, "COMBINED_config_6.json"), os.path.join(rc_path, "RL_config_66.json"), False, False)
r_p_i = get_result_config(os.path.join(rr_path, "COMBINED_config_402.json"), os.path.join(rc_path, "RL_config_402.json"), False, False)
r_b_i = get_result_config(os.path.join(rr_path, "COMBINED_config_411.json"), os.path.join(rc_path, "RL_config_411.json"), False, False)

tr_res_combined.extend([r_m, r_m_i, r_p, r_p_i, r_b, r_b_i])

In [69]:
tr_res_combined = pd.DataFrame(tr_res_combined, columns=['Configuration', 'bleu_1', 'bleu_2', 'bleu_3', 'bleu_4', "ROUGE-L",
                                      "Novelty", "Distinctness", "Duplicateness", "Vocabulary", "Number of duplicate captions",
                                      "Number of distinct captions", "Number of within item duplicates", "Vocabulary size predictions",
                                      "Vocabulary size references", "Vocabulary percentage preds / references", "Vocabulary percentage train / references"])
tr_res_combined

,Configuration,bleu_1,bleu_2,bleu_3,bleu_4,ROUGE-L,Novelty,Distinctness,Duplicateness,Vocabulary,Number of duplicate captions,Number of distinct captions,Number of within item duplicates,Vocabulary size predictions,Vocabulary size references,Vocabulary percentage preds / references,Vocabulary percentage train / references
0,RL_config_601,0.342417,0.251446,0.211509,0.189053,0.306642,0.188358,0.306881,0.435503,0.925018,7686,3403,6786,3812,4121,0.548884,0.593377
1,RL_config_66,0.331010,0.235932,0.194536,0.171593,0.284740,0.332820,0.295259,0.501669,0.856103,7225,3027,7817,3528,4121,0.507991,0.593377
2,RL_config_602,0.343051,0.250762,0.210007,0.187067,0.305338,0.369336,0.442102,0.422988,0.925989,6244,4948,6591,3816,4121,0.549460,0.593377
3,RL_config_402,0.339043,0.248022,0.207266,0.184103,0.307577,0.556155,0.600593,0.356758,0.919922,4713,7087,5559,3791,4121,0.545860,0.593377
4,RL_config_603,0.305334,0.214131,0.174789,0.153057,0.278830,0.256001,0.287701,0.482416,0.855618,7477,3020,7517,3526,4121,0.507703,0.593377
5,RL_config_411,0.337996,0.245930,0.205458,0.182826,0.298381,0.253819,0.347002,0.415159,0.951225,7373,3918,6469,3920,4121,0.564435,0.593377


In [67]:
tr_res_combined.to_excel(r"C:\Users\s159655\Documents\JADS\Thesis\RESULTS\Results_Paper\IMPROVING RESULTS\Trainable_emb.xlsx")

In [59]:
# Get results for respective configurations;  6 , 402 and 411
results = []
def compare_full(configs_new, configs_new_2, name):
    """
    configs_new: merge, parinject and brownlee config (in that order)
    """
    results = []
    for w in ["UO", "AE", "MA", "RL"]:
        if w == "UO":
            pa = path_res_uo
        if w == "AE":
            pa = path_res_ae 
        if w == "MA":
            pa = path_res_ma
        if w == "RL":
            pa = path_res_rl
            with open(os.path.join(pa, f"results_dict_{w}_config_66.json"), 'r') as f:
                merge = json.load(f)
        else:
            with open(os.path.join(pa, f"results_dict{w}_config_6.json"), 'r') as f:
                merge = json.load(f)
        with open(os.path.join(pa, f"results_dict{w}_{configs_new[0]}"), 'r') as f:
            merge_rob = json.load(f)
        with open(os.path.join(pa, f"results_dict{w}_config_402.json"), 'r') as f:
            par = json.load(f)
        with open(os.path.join(pa, f"results_dict{w}_{configs_new[1]}"), 'r') as f:
            par_rob = json.load(f)
        if w == "RL":
            with open(os.path.join(pa, f"results_dict{w}_config_451.json"), 'r') as f:
                brown = json.load(f)
        else:
            with open(os.path.join(pa, f"results_dict{w}_config_411.json"), 'r') as f:
                brown = json.load(f)
        with open(os.path.join(pa, f"results_dict{w}_{configs_new[2]}"), 'r') as f:
            brown_rob = json.load(f)
            
        # load image_cat combined models
        with open(os.path.join(pa, f"results_dict{w}_{configs_new_2[0]}"), 'r') as f:
            merge_imgcat = json.load(f)
        with open(os.path.join(pa, f"results_dict{w}_{configs_new_2[1]}"), 'r') as f:
            par_imgcat = json.load(f)
        with open(os.path.join(pa, f"results_dict{w}_{configs_new_2[2]}"), 'r') as f:
            brown_imgcat = json.load(f)

        results.append((f"{w} merge - image", merge["BLEU_SCORES"]["BLEU_1"], merge["BLEU_SCORES"]["BLEU_2"], 
                        merge["BLEU_SCORES"]["BLEU_3"], merge["BLEU_SCORES"]["BLEU_4"], 
                        merge["ROUGE_SCORES"]["Avg"]["rouge-l"]["f"], 
                    merge["novelty_score"], merge["fraction_Distinct"], merge["fraction_within_duplicates"], merge["vocab_perc_intrain"]))
        results.append((f"{w} merge - {name}",
                    merge_rob["BLEU_SCORES"]["BLEU_1"], merge_rob["BLEU_SCORES"]["BLEU_2"], 
                        merge_rob["BLEU_SCORES"]["BLEU_3"], merge_rob["BLEU_SCORES"]["BLEU_4"],  merge_rob["ROUGE_SCORES"]["Avg"]["rouge-l"]["f"], 
                    merge_rob["novelty_score"], merge_rob["fraction_Distinct"], merge_rob["fraction_within_duplicates"], merge_rob["vocab_perc_intrain"]))
        
        results.append((f"{w} merge - img + cat",
                    merge_imgcat["BLEU_SCORES"]["BLEU_1"], merge_imgcat["BLEU_SCORES"]["BLEU_2"], 
                        merge_imgcat["BLEU_SCORES"]["BLEU_3"], merge_imgcat["BLEU_SCORES"]["BLEU_4"],  merge_imgcat["ROUGE_SCORES"]["Avg"]["rouge-l"]["f"], 
                    merge_imgcat["novelty_score"], merge_imgcat["fraction_Distinct"], merge_imgcat["fraction_within_duplicates"], merge_imgcat["vocab_perc_intrain"]))
        
        results.append((f"{w} parinject - image", par["BLEU_SCORES"]["BLEU_1"], par["BLEU_SCORES"]["BLEU_2"], 
                        par["BLEU_SCORES"]["BLEU_3"], par["BLEU_SCORES"]["BLEU_4"],
                        par["ROUGE_SCORES"]["Avg"]["rouge-l"]["f"], 
                        par["novelty_score"], par["fraction_Distinct"], par["fraction_within_duplicates"], par["vocab_perc_intrain"]))

        results.append((f"{w} parinject - {name}", par_rob["BLEU_SCORES"]["BLEU_1"], par_rob["BLEU_SCORES"]["BLEU_2"], 
                        par_rob["BLEU_SCORES"]["BLEU_3"], par_rob["BLEU_SCORES"]["BLEU_4"], par_rob["ROUGE_SCORES"]["Avg"]["rouge-l"]["f"], 
                        par_rob["novelty_score"], par_rob["fraction_Distinct"], par_rob["fraction_within_duplicates"], par_rob["vocab_perc_intrain"]))

        results.append((f"{w} brownlee - image", brown["BLEU_SCORES"]["BLEU_1"], brown["BLEU_SCORES"]["BLEU_2"], 
                        brown["BLEU_SCORES"]["BLEU_3"], brown["BLEU_SCORES"]["BLEU_4"], brown["ROUGE_SCORES"]["Avg"]["rouge-l"]["f"], 
                        brown["novelty_score"], brown["fraction_Distinct"], brown["fraction_within_duplicates"], brown["vocab_perc_intrain"]))

        results.append((f"{w} brownlee - {name}", brown_rob["BLEU_SCORES"]["BLEU_1"], brown_rob["BLEU_SCORES"]["BLEU_2"], 
                        brown_rob["BLEU_SCORES"]["BLEU_3"], brown_rob["BLEU_SCORES"]["BLEU_4"], brown_rob["ROUGE_SCORES"]["Avg"]["rouge-l"]["f"], 
                        brown_rob["novelty_score"], brown_rob["fraction_Distinct"], brown_rob["fraction_within_duplicates"], brown_rob["vocab_perc_intrain"]))
    
    return results

In [60]:
results_trainable_embeddings = compare_full(["config_601.json", "config_602.json", "config_603.json"], "emb_tr")

In [62]:
results_trainable_embeddings = pd.DataFrame(results_trainable_embeddings, columns=["Architecture", "BLEU-1", "BLEU-2", "BLEU-3", "BLEU-4", "ROUGE-L", "Novelty", "Distinctness", "Duplicateness", "Vocabulary"])
results_trainable_embeddings

,Architecture,BLEU-1,BLEU-2,BLEU-3,BLEU-4,ROUGE-L,Novelty,Distinctness,Duplicateness,Vocabulary
0,UO merge - image,0.312579,0.188071,0.129381,0.097371,0.259068,0.618038,0.471627,0.423575,0.837804
1,UO merge - emb_tr,0.313071,0.191538,0.135407,0.104948,0.261192,0.256639,0.344210,0.411431,0.929170
2,UO parinject - image,0.315605,0.191763,0.132281,0.099004,0.268908,0.856056,0.754383,0.273802,0.904862
3,UO parinject - emb_tr,0.314466,0.193321,0.136207,0.105001,0.264396,0.518297,0.534351,0.370142,0.932104
4,UO brownlee - image,0.317931,0.194015,0.136266,0.105011,0.261253,0.379696,0.391095,0.379048,0.937133
5,UO brownlee - emb_tr,0.316671,0.195688,0.139323,0.108838,0.263820,0.301166,0.345750,0.418232,0.931685
6,AE merge - image,0.391545,0.352578,0.334460,0.323253,0.363984,0.037407,0.096510,0.702537,0.849810
7,AE merge - emb_tr,0.407166,0.370725,0.353781,0.343450,0.395393,0.044069,0.186621,0.630028,0.930608
8,AE parinject - image,0.398013,0.361703,0.344602,0.333861,0.392885,0.149116,0.333812,0.612862,0.932510
9,AE parinject - emb_tr,0.402250,0.364519,0.346802,0.335814,0.398872,0.110684,0.276535,0.615168,0.929658


In [63]:
results_trainable_embeddings.to_excel(r"C:\Users\s159655\Documents\JADS\Thesis\RESULTS\Results_Paper\IMPROVING RESULTS\trainable_emb_full.xlsx")

# Threshold sizes

In [45]:
# threshold 10:
exclude_configs = list(range(0, 109)) + list(range(155,1000))
tmp = get_results(path_res_rl, path_conf_rl, exclude_configs)
tmp

,Configuration,Embedding,Feature extractor,Architecture,Optimizer,Dropout,Nr. nodes,BLEU-1,BLEU-2,BLEU-3,...,Distinctness,Duplicateness,Vocabulary,Number of duplicate captions,Number of distinct captions,Number of within item duplicates,Vocabulary size predictions,Vocabulary size references,Vocabulary percentage preds / references,Vocabulary percentage train / references
0,RL_config_109,glove_300d_wiki_thr10_emb.p,incv3_fromscratch,merge,adam,NOT_REALLY_TRUE,64,0.321358,0.242678,0.213968,...,0.343840,0.517634,0.441518,1145,600,1453,570,1291,0.284006,0.643249
1,RL_config_110,glove_300d_wiki_thr10_emb.p,incv3_fromscratch,merge,adam,NOT_REALLY_TRUE,256,0.350952,0.271490,0.241922,...,0.274386,0.586035,0.526723,1153,436,1645,680,1291,0.338814,0.643249
2,RL_config_111,glove_300d_wiki_thr10_emb.p,incv3_fromFF,merge,adam,False,64,0.316456,0.227264,0.194634,...,0.368865,0.394015,0.395817,1273,744,1106,511,1291,0.254609,0.643249
3,RL_config_112,glove_300d_wiki_thr10_emb.p,incv3_fromFF,merge,adam,False,256,0.344311,0.265290,0.235161,...,0.124410,0.572141,0.592564,1485,211,1606,765,1291,0.381166,0.643249
4,RL_config_113,glove_300d_wiki_thr10_emb.p,incv3_fromFF,merge,adam,NOT_REALLY_TRUE,64,0.298367,0.222978,0.195757,...,0.316369,0.459922,0.398141,1303,603,1291,514,1291,0.256104,0.643249
5,RL_config_114,glove_300d_wiki_thr10_emb.p,incv3_fromFF,merge,adam,NOT_REALLY_TRUE,256,0.333149,0.259119,0.230838,...,0.141827,0.585322,0.570875,1428,236,1643,737,1291,0.367215,0.643249
6,RL_config_115,glove_300d_wiki_thr10_emb.p,incv3_noft,merge,adam,False,64,0.328174,0.247284,0.217634,...,0.213554,0.528678,0.347792,1381,375,1484,449,1291,0.223717,0.643249
7,RL_config_116,glove_300d_wiki_thr10_emb.p,incv3_noft,merge,adam,False,256,0.335113,0.259033,0.229881,...,0.156730,0.600285,0.583269,1372,255,1685,753,1291,0.375187,0.643249
8,RL_config_117,glove_300d_wiki_thr10_emb.p,incv3_noft,merge,adam,NOT_REALLY_TRUE,64,0.284966,0.206930,0.179254,...,0.296540,0.502316,0.406662,1281,540,1410,525,1291,0.261584,0.643249
9,RL_config_118,glove_300d_wiki_thr10_emb.p,incv3_noft,merge,adam,NOT_REALLY_TRUE,256,0.345879,0.266710,0.237301,...,0.262078,0.632348,0.517428,1115,396,1775,668,1291,0.332835,0.643249


# Image examples

In [15]:
ip = r"C:\Users\s159655\Documents\JADS\Thesis\Code\Train_Test\results_dict_imgs.json"
with open(ip, 'r') as f:
    img_res = json.load(f)

imgs_dict = {}
    
for key, value in img_res.items():
    if key[:2] != "MA":
        continue
    else:
        for ix, item in enumerate(value["img_names"]):
            if not item in imgs_dict.keys():
                imgs_dict[item] = {}
                imgs_dict[item]["preds"] = []
            imgs_dict[item]["preds"].append(value["PREDICTIONS"][ix])
            imgs_dict[item]["ref"] = value["REFERENCES"][ix]


In [16]:
imgs_dict

{'MW_J8982_EG9169_2.p': {'preds': ['minimalist and modern these wearwitheverything flats in soft supple leather have squaredoff shape we love these might just be the easiest coolgirl shoes ever cushiness alert weve added an extra layer of padding to make them even comfier',
   'inspired by art these intricately knotted suede sandals are the lowkey statement shoe your favorite has been',
   'for over years this sandal has been without an equal when it comes to and cool styles you can in for these suede clogs combine plush shearling with its signature footbed',
   'made flats slides proves these day leather alert are alert alert alert alert alert alert alert alert alert alert alert alert alert alert alert alert alert alert in in in pair pair alert alert alert in in rainbow rainbow alert alert alert alert alert in in rainbow rainbow alert alert alert alert alert in in rainbow rainbow alert alert alert alert alert in in rainbow rainbow alert alert alert alert in in rainbow rainbow alert do

In [20]:
imgs_dict_uo = {}
    
for key, value in img_res.items():
    if key[:2] != "UO":
        continue
    else:
        for ix, item in enumerate(value["img_names"]):
            if not item in imgs_dict_uo.keys():
                imgs_dict_uo[item] = {}
                imgs_dict_uo[item]["preds"] = []
            imgs_dict_uo[item]["preds"].append(value["PREDICTIONS"][ix])
            imgs_dict_uo[item]["ref"] = value["REFERENCES"][ix]

In [21]:
imgs_dict_uo

{'UO_52022092_070_1.p': {'preds': ['add an edge with this necklace by uo featuring mixed chains in chunky silhouette complete with joint clasp closure and extender chain',
   'add some texture to your look with this choker necklace by urban outfitters classic metal chains that rests high on the décolletage finished with lobster clasp closure extender chain for adjustability',
   'double your of on style with this dual coin necklace by uo two metal chains make layering easy with an astrology coin pendent complete with lobster clasp extender chain',
   'classic chain chain by by pendant brass with for closure wear'],
  'ref': ['show off your legs with this essential anklet set for uo which includes chains in snake figaro and classic construction wear them alone in twos or all together for an accessorized finish to your summer look each complete with an adjuster chain for size variance']},
 'UO_52761236_029_1.p': {'preds': ['if scarf provide cushy comfort do you really want it the stella 

In [19]:
img_res

{'RL_config_66.json': {'BLEU_SCORES': [],
  'ROUGE_SCORES': [],
  'IMG_IDS': [],
  'PREDICTIONS': ['this reissue of the polo sport waist pack has been recreated to the size and scale of the original and features authentic of the rubberised polo sport label and zip pulls constructed from durable nylon and with waterresistant base the design offers storage',
   'an american style standard since the polo shirt has been imitated but never matched over the decades ralph lauren has reimagined his signature style in wide array of colours and fits yet all retain the quality and attention to detail of the iconic original this trim version is made from our highly breathable cotton mesh which offers textured look and soft feel',
   'an american style standard since the polo shirt has been imitated but never matched over the decades ralph lauren has reimagined his signature style in wide array of colours and fits yet all retain the quality and attention to detail of the iconic original this trim v

# Improving models

In [31]:
def get_beam_result(results, confname):
    bleu = results['BLEU_SCORES']
    bleu_1 = bleu['BLEU_1']
    bleu_2 = bleu['BLEU_2']
    bleu_3 = bleu['BLEU_3']
    bleu_4 = bleu['BLEU_4']

    # Main novelty measures
    novelty_percentage = results["novelty_score"] 
    fraction_distinct = results["fraction_Distinct"]
    within_item_duplicates = results["within_item_duplicates"]
    fraction_within_item_duplicates = results["fraction_within_duplicates"]
    vocab_percentage_from_train = results["vocab_perc_intrain"]

    # Secondary novelty measures
    nr_duplicates = results["nr_duplicate_total"]
    nr_distinct = results["nr_distinct"]
    vocab_size_predictions = results["vocab_size_predictions"]
    vocab_size_references = results["vocab_size_ref"]
    vocab_percentage_inref = results["vocab_perc_inref"]
    vocab_percentage_ref_intrain = results["vocab_per_test_train"]




    rouge_l = results["ROUGE_SCORES"]["Avg"]["rouge-l"]["f"]
    return [confname, bleu_1, bleu_2, bleu_3, bleu_4, rouge_l,
                      novelty_percentage, fraction_distinct, fraction_within_item_duplicates,
                      vocab_percentage_from_train, nr_duplicates, nr_distinct, within_item_duplicates,
                        vocab_size_predictions, 
                      vocab_size_references, vocab_percentage_inref, vocab_percentage_ref_intrain]

In [39]:
# Beam width
beam_res = []
p_res_base = r"C:\Users\s159655\Documents\JADS\Thesis\RESULTS\Results_Paper\ma_results\all_results"
p_conf_base = r"C:\Users\s159655\Documents\JADS\Thesis\Code\Train_Test\Configurations\MA_configs"
for config in [6, 402, 411]:
    for k in [1, 3, 5, 7, 9, 10, 11, 20]:

        pr = os.path.join(p_res_base, f"results_dictMA_config_{config}_beam{k}.json")
        if not os.path.exists(pr):
            continue
        with open(pr, 'r') as f:
            results = json.load(f)
        res = get_beam_result(results, f"MA_config_{config}_beam{k}")
        beam_res.append(res)
    pr = os.path.join(p_res_base, f"results_dictMA_config_{config}.json")
    with open(pr, 'r') as f:
        results = json.load(f)
    res = get_beam_result(results, f"MA_config_{config}")
    beam_res.append(res)



beam_res = pd.DataFrame(beam_res, columns=['Configuration', 'BLEU-1', 'BLEU-2', 'BLEU-3', 'BLEU-4', "ROUGE-L",
                                  "Novelty", "Distinctness", "Duplicateness", "Vocabulary", "Number of duplicate captions",
                                  "Number of distinct captions", "Number of within item duplicates", "Vocabulary size predictions",
                                  "Vocabulary size references", "Vocabulary percentage preds / references", "Vocabulary percentage train / references"])

In [41]:
beam_res.to_excel(r"C:\Users\s159655\Documents\JADS\Thesis\RESULTS\Results_Paper\IMPROVING RESULTS\beam_results.xlsx")

# Bounding boxes and attribute-included

In [32]:
config_numbers_a = list(range(901, 910))
res_paths = []
for i in config_numbers_a:
    res_paths.append(os.path.join(r"C:\Users\s159655\Documents\JADS\Thesis\RESULTS\Results_Paper\ma_results\all_results", 
                                  f"results_dictMA_config_{i}.json"))
for i in [6, 402, 411]:
    res_paths.append(os.path.join(r"C:\Users\s159655\Documents\JADS\Thesis\RESULTS\Results_Paper\ma_results\all_results", 
                                  f"results_dict_cropped_MA_config_{i}.json"))

df_bb = get_results(res_paths, path_conf_ma)

MA_config_901.json
MA_config_902.json
MA_config_903.json
MA_config_904.json
MA_config_905.json
MA_config_906.json
MA_config_907.json
MA_config_908.json
MA_config_909.json
MA_config_6.json
MA_config_402.json
MA_config_411.json


In [33]:
df_bb

,Configuration,Embedding,Feature extractor,Architecture,Optimizer,Dropout,Nr. nodes,BLEU-1,BLEU-2,BLEU-3,...,Distinctness,Duplicateness,Vocabulary,Number of duplicate captions,Number of distinct captions,Number of within item duplicates,Vocabulary size predictions,Vocabulary size references,Vocabulary percentage preds / references,Vocabulary percentage train / references
0,MA_config_901,glove_300d_wiki_thr5_emb.p,incv3_noft,merge,adam,False,256,0.301611,0.220832,0.190313,...,0.364411,0.243557,0.889445,1268,727,567,1778,1999,0.611207,0.687178
1,MA_config_902,glove_300d_wiki_thr5_emb.p,incv3_fromFF,merge,adam,False,256,0.327543,0.246688,0.215261,...,0.401228,0.269759,0.893947,1170,784,628,1787,1999,0.614300,0.687178
2,MA_config_903,glove_300d_wiki_thr5_emb.p,incv3_fromscratch,merge,adam,False,256,0.306058,0.226970,0.196874,...,0.426181,0.223797,0.905953,1166,866,521,1811,1999,0.622551,0.687178
3,MA_config_904,glove_300d_wiki_thr5_emb.p,incv3_noft,parinject,adam,False,256,0.308595,0.230264,0.200399,...,0.523364,0.213058,0.821411,969,1064,496,1642,1999,0.564455,0.687178
4,MA_config_905,glove_300d_wiki_thr5_emb.p,incv3_fromFF,parinject,adam,False,256,0.332541,0.251081,0.219130,...,0.602787,0.222938,0.898949,798,1211,519,1797,1999,0.617738,0.687178
5,MA_config_906,glove_300d_wiki_thr5_emb.p,incv3_fromscratch,parinject,adam,False,256,0.316999,0.237730,0.207093,...,0.605461,0.153351,0.897949,838,1286,357,1795,1999,0.617051,0.687178
6,MA_config_907,fasttext_300d_wiki_thr5_emb.p,incv3_noft,brownlee,adam,False,256,0.316665,0.234143,0.202453,...,0.419144,0.247423,0.929965,1153,832,576,1859,1999,0.639051,0.687178
7,MA_config_908,fasttext_300d_wiki_thr5_emb.p,incv3_fromFF,brownlee,adam,False,256,0.319188,0.235593,0.203391,...,0.464965,0.242698,0.921961,1069,929,565,1843,1999,0.633551,0.687178
8,MA_config_909,fasttext_300d_wiki_thr5_emb.p,incv3_fromscratch,brownlee,adam,False,256,0.296750,0.216211,0.185926,...,0.396226,0.262457,0.908954,1184,777,611,1817,1999,0.624613,0.687178
9,MA_config_6,glove_300d_wiki_thr5_emb.p,incv3_fromFF,merge,adam,False,256,0.295747,0.213160,0.182050,...,0.216550,0.337629,0.807404,1458,403,786,1614,1999,0.554830,0.687178


In [34]:
df_bb.to_excel(r"C:\Users\s159655\Documents\JADS\Thesis\RESULTS\Results_Paper\IMPROVING RESULTS\Bounding_Boxes.xlsx")

In [99]:
def get_results_ae(path_res, path_conf, exclude_configs=[]):
    """
    path_res: main path to results folder
    path_conf: main path to config folder
    exclude_configs: list of configuration number to exclude """
    df = []
    if type(path_res) == list:
        paths = path_res
    else:
        paths = [os.path.join(path_res, x) for x in os.listdir(path_res)]
    for path in paths:
        # Skip if config number is excluded
        config_nr = path.split("_")[-1][:-5]
        try:
            if int(config_nr) in exclude_configs:
                continue
        except ValueError:
            print(f"ValueError: {config_nr} is not a number")
            continue
        if path.endswith(".json"):
            if "cropped" in path:
                confname = os.path.basename(path)[21:]
            else:
                confname = os.path.basename(path)[12:]
            with open(path, 'r') as f:
                results = json.load(f)
            try:
                with open(os.path.join(path_conf, confname), 'r') as f2:
                    conf = json.load(f2)
            except FileNotFoundError:
                try: 
                    # For some older files, the config name is slightly different
                    confname = confname[1:]
                    with open(os.path.join(path_conf, confname), 'r') as f2:
                        conf = json.load(f2)
                except FileNotFoundError as e:
                    print(str(e))
                    continue
            try:
                bleu = results['BLEU_SCORES']
                bleu_1 = bleu['BLEU_1']
                bleu_2 = bleu['BLEU_2']
                bleu_3 = bleu['BLEU_3']
                bleu_4 = bleu['BLEU_4']

                # Main novelty measures
                novelty_percentage = results["novelty_score"] 
                fraction_distinct = results["fraction_Distinct"]
                within_item_duplicates = results["within_item_duplicates"]
                fraction_within_item_duplicates = results["fraction_within_duplicates"]
                vocab_percentage_from_train = results["vocab_perc_intrain"]

                # Secondary novelty measures
                nr_duplicates = results["nr_duplicate_total"]
                nr_distinct = results["nr_distinct"]
                vocab_size_predictions = results["vocab_size_predictions"]
                vocab_size_references = results["vocab_size_ref"]
                vocab_percentage_inref = results["vocab_perc_inref"]
                vocab_percentage_ref_intrain = results["vocab_per_test_train"]




                rouge_l = results["ROUGE_SCORES"]["Avg"]["rouge-l"]["f"]

                image_input = conf["image_input"]
                feat_ex = conf['img_folder']
                arc = conf['architecture_type']
            except KeyError as e:
                print(str(e), confname) 

            df.append((confname[:-5], image_input, feat_ex, arc, bleu_1, bleu_2, bleu_3, bleu_4, rouge_l,
                      novelty_percentage, fraction_distinct, fraction_within_item_duplicates,
                      vocab_percentage_from_train, nr_duplicates, nr_distinct, within_item_duplicates,
                        vocab_size_predictions, 
                      vocab_size_references, vocab_percentage_inref, vocab_percentage_ref_intrain))
    df = pd.DataFrame(df, columns=['Configuration', 'Image Included', 'Feature extractor', 'Architecture', 'BLEU-1', 'BLEU-2', 'BLEU-3', 'BLEU-4', "ROUGE-L",
                                  "Novelty", "Distinctness", "Duplicateness", "Vocabulary", "Number of duplicate captions",
                                  "Number of distinct captions", "Number of within item duplicates", "Vocabulary size predictions",
                                  "Vocabulary size references", "Vocabulary percentage preds / references", "Vocabulary percentage train / references"])
    return df

In [100]:
# Attribute enriched
config_numbers_a = list(range(1001, 1010))
res_paths = []
for i in config_numbers_a:
    res_paths.append(os.path.join(r"C:\Users\s159655\Documents\JADS\Thesis\RESULTS\Results_Paper\ma_results\all_results", 
                                  f"results_dictMA_config_{i}.json"))
res_ae = get_results_ae(res_paths, path_conf_ma)
res_ae.to_excel(r"C:\Users\s159655\Documents\JADS\Thesis\RESULTS\Results_Paper\IMPROVING RESULTS\Attribute_Enriched.xlsx")

# Preliminary experiment 3

In [52]:
paths_prel3 = []
for i in range(1, 18):
    paths_prel3.append(os.path.join(r"C:\Users\s159655\Documents\JADS\Thesis\RESULTS\Results_Preliminary\Prelim_2", 
                                   f"results_dict_RL_test_config_{i}.json"))
path_conf_rl_prelim = r"C:\Users\s159655\Documents\JADS\Thesis\Code\Train_Test\Configurations\RL_test_configs"
df_pr3 = get_results(paths_prel3, path_conf_rl_prelim)

In [54]:
df_pr3.to_excel(r"C:\Users\s159655\Documents\JADS\Thesis\RESULTS\Results_Paper\prelim_3.xlsx")

# Preliminary Study 2

In [65]:
# Add vs concat
prelim1_path = r"C:\Users\s159655\Documents\JADS\Thesis\RESULTS\Results_Preliminary\prelim1"
prelimb_path = r"C:\Users\s159655\Documents\JADS\Thesis\RESULTS\Results_Preliminary\Prelim_B"
results_per_epoch = []
for feat in ["incv3", "resnet50"]:
    for a in ["concat", "add"]:
        with open(os.path.join(prelim1_path, f"RL_{a}_{feat}_ft_results.json"), 'r') as f:
            add_r = json.load(f)

            bleu_scores = add_r["bleu_scores"]
 
            for i in bleu_scores:
                results_per_epoch.append((feat, a, i[0], i[1][0], i[1][1], i[1][2], i[1][3]))

results_per_epoch = pd.DataFrame(results_per_epoch, columns=["Extractor", "Concat / Add", "Nr. Epochs", "BLEU-1", 
                                                             "BLEU-2", "BLEU-3", "BLEU-4"])
                
results_per_epoch.to_excel(os.path.join(prelimb_path, "Epochs and concat - add.xlsx"))            
                
results_per_epoch

,Extractor,Concat / Add,Nr. Epochs,BLEU-1,BLEU-2,BLEU-3,BLEU-4
0,incv3,concat,2,0.278961,0.176085,0.146477,0.135092
1,incv3,concat,4,0.292442,0.193853,0.165007,0.155815
2,incv3,concat,6,0.297258,0.202163,0.175628,0.166849
3,incv3,concat,8,0.280137,0.184515,0.159236,0.149066
4,incv3,add,2,0.274166,0.168258,0.135113,0.125191
5,incv3,add,4,0.296147,0.196015,0.167488,0.157777
6,incv3,add,6,0.288124,0.198358,0.169644,0.160133
7,incv3,add,8,0.276682,0.176844,0.150405,0.140542
8,resnet50,concat,2,0.247408,0.147579,0.119482,0.110424
9,resnet50,concat,4,0.250461,0.158444,0.136765,0.127958


In [5]:
# Brownlee and par-inject (indicated by BASE)
results_per_epoch_2 = []
prelim1_path = r"C:\Users\s159655\Documents\JADS\Thesis\RESULTS\Results_Preliminary\prelim1"
prelimb_path = r"C:\Users\s159655\Documents\JADS\Thesis\RESULTS\Results_Preliminary\Prelim_B"

for arch in ["BASE", "BROWNLEE"]:
       
    for a in range(1, 5):
        if (a > 3) & (arch == "BROWNLEE"):
            break
        # Get config
        with open(os.path.join(r"C:\Users\s159655\Documents\JADS\Thesis\Code\Train_Test\Configurations\rl_prelim_configs",
                          f"RL_configuration_{arch}_{a}.json"), "r") as f:
            conf = json.load(f)
        
        with open(os.path.join(prelim1_path, f"RL_PI_{arch}_CONF{a}_evaluations.json"), 'r') as f:
            add_r = json.load(f)

        bleu_scores = add_r["bleu_scores"]
        
        extr = conf["feature_extractor"]
        opt = conf["optimizer"]
        nr_nodes = conf["nr_nodes"]
        dropout = conf["dropout"]
        print(nr_nodes)
        
        for i in bleu_scores:
            if arch == "BASE":
                feat = "Par-inject"
            else:
                feat = arch

            results_per_epoch_2.append((feat, i[0], extr, opt, nr_nodes, dropout, i[1][0], i[1][1], i[1][2], i[1][3]))

results_per_epoch_2 = pd.DataFrame(results_per_epoch_2, columns=["Arch", "Nr. Epochs", "Extractor", "Opt.", "Nodes", "Dropout", "BLEU-1", 
                                                             "BLEU-2", "BLEU-3", "BLEU-4"])

64
256
256
256
64
256
256


In [7]:
results_per_epoch_2.to_excel(os.path.join(prelimb_path, "Epochs parinject.xlsx"))

In [61]:
good_preds

[5,
 ['this handsome leather backpack features shape for look thats unmistakably polo',
  'this handsome leather backpack features pattern inspired by the of the polo sport logo and interior',
  'this handsome leather is accented with ralph laurens signature embroidered pony',
  'this handsome leather belt features shape inspired by the',
  'sleek fabric and design give this jacket modern polo upgrade',
  'made from lightweight cotton denim this is finished with black and striped cast and leather',
  'made from lightweight cotton denim this is lined with black and striped rl and two pockets and leather',
  'made from lightweight cotton denim this is lined with black and rivets interior leather',
  'this handsome leather backpack features pattern inspired by the of the polo sport logo and interior interior and the signature polo sport collection from the early',
  'this handsome leather belt features metal interior and leather panels that is finished with ralph laurens signature embroid

# Prep example

In [97]:
import pickle
with open("MA_002082_3.p", 'rb') as f:
    img_feat = pickle.load(f)

In [98]:
img_feat

array([[0.04618475, 0.06508254, 0.08884629, ..., 0.38656008, 0.36738274,
        1.5090187 ]], dtype=float32)

# Generate LaTex code for description examples file|